In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time

import os

# Data Processing

In [2]:
date_number_list = [3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

dates_list = {}
for date_count, date_number in enumerate(date_number_list):
    dates_list[date_count] = '9/' + str(date_number_list[date_count]) + '/2024 8:00'

type(dates_list[1])

str

In [3]:
directory_path = '../data/pems_flow___101_N_Sep_to_Nov_2024/'
file_name = '001___402376_Palo_Alto___main.xlsx'

# df = pd.read_excel(directory_path + file_name)
df = pd.read_excel(directory_path + file_name, index_col=0)

# print(df.to_string())
print()

# print(df.loc['2024-09-03 08:00:00', 'Flow (Veh/Hour)'])
print(df.loc['9/3/2024 8:00', 'Flow (Veh/Hour)'])



4520.0


In [7]:
start_time = time.time()

directory_path = '../data/pems_flow___101_N_Sep_to_Nov_2024/'
file_name_list = [f for f in os.listdir(directory_path) \
                  if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x' and f[0] != "~"]
file_name_list.sort()

# print("file_name_list:", file_name_list)
# print()

# file_name = '001___402376_Palo_Alto___main.xlsx'

date_number_list_Sep = ['03', '04', '05', '06', '09', '10', '11', '12', '13', '16', \
                        '17', '18', '19', '20', '23', '24', '25', '26', '27', '30']
date_number_list_Oct = ['01', '02', '03', '04', '07', '08', '09', '10', '11', '14', \
                        '15', '16', '17', '18', '21', '22', '23', '24', '25', '28', \
                        '29', '30', '31']
date_number_list_Nov = ['01', '04', '05', '06', '07', '08', '12', '13', '14', '15', \
                        '18', '19', '20', '21', '22']

hours_list = ['06', '07', '08', '09', '10', '11', '12']


times_dict = {}
counter = 0
for date_number in date_number_list_Sep:
    for hour in hours_list:
        times_dict[counter] = '2024-09-' + date_number + ' ' + hour + ':00:00'
        counter += 1
for date_number in date_number_list_Oct:
    for hour in hours_list:
        times_dict[counter] = '2024-10-' + date_number + ' ' + hour + ':00:00'
        counter += 1
for date_number in date_number_list_Nov:
    for hour in hours_list:
        times_dict[counter] = '2024-11-' + date_number + ' ' + hour + ':00:00'
        counter += 1

dict_data = {}
dict_data["Times / Type"] = ["Type"] + list(times_dict.values())

# print()
# print("times_dict.values():", times_dict.values())
# print("dict_data[Times / Type]:", dict_data["Times / Type"])
# print()

counter = 1
for file_name in file_name_list:
    print("counter:", counter)
    counter += 1
    print("file_name:", file_name)
    
    if file_name[-5:] != ".xlsx":
        assert 1 == 0, "This case should not happen, since all files must be of type .xlsx"
        
    dict_data[file_name] = []
    
    if file_name[-9:-5] == "main":
        dict_data[file_name] += ["main"]
    elif file_name[-7:-5] == "on":
        dict_data[file_name] += ["on"]
    elif file_name[-8:-5] == "off":
        dict_data[file_name] += ["off"]
    else:
        assert 1 == 0, "This case should not happen, since all file names must end with main, on, or off."

    df = pd.read_excel(directory_path + file_name, index_col = 0)
    
    for date_number in date_number_list_Sep:
#         print("date_number:", date_number)

        for hour in hours_list:
#             if date_number == '03' and hour == '06':
#                 print("hour:", hour)
#                 print("df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']:", \
#                      df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)'])
#                 print("type(df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']):", \
#                      type(df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']))

            dict_data[file_name] += [df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']]
    
    for date_number in date_number_list_Oct:
        for hour in hours_list:
            dict_data[file_name] += [df.loc['2024-10-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']]
    
    for date_number in date_number_list_Nov:
        for hour in hours_list:
            dict_data[file_name] += [df.loc['2024-11-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)']]

print("dict_data:", dict_data)

df_dict_data = pd.DataFrame(dict_data)
df_dict_data.to_csv(directory_path + 'data_filtered.csv', index = False)

# print(df.loc['2024-09-03 ' + hour + ':00:00', 'Flow (Veh/Hour)'])
# print(df.loc['2024-09-' + date_number + ' ' + hour + ':00:00', 'Flow (Veh/Hour)'])


# f = open("../data/pems_flow___101_N_Sep_to_Nov_2024/data_filtered.txt", "w+")
# f.write(json.dumps(dict_data, indent=4, sort_keys=True))



# # df = pd.read_excel(directory_path + file_name)
# df = pd.read_excel(directory_path + file_name, index_col=0)

# # print(df.to_string())
# print()

# # print(df.loc['2024-09-03 08:00:00', 'Flow (Veh/Hour)'])
# print(df.loc['9/3/2024 8:00', 'Flow (Veh/Hour)'])


end_time = time.time()

print()
print("Time:", end_time - start_time)


counter: 1
file_name: 001___402376_Palo_Alto___main.xlsx
counter: 2
file_name: 002___402377_Palo_Alto___main.xlsx
counter: 3
file_name: 003___402379_Palo_Alto___main.xlsx
counter: 4
file_name: 004___425696_Palo_Alto___off.xlsx
counter: 5
file_name: 005___425697_Palo_Alto___main.xlsx
counter: 6
file_name: 006___425897_Palo_Alto___on.xlsx
counter: 7
file_name: 007___402380_Palo_Alto___main.xlsx
counter: 8
file_name: 008___404529_East_Palo_Alto___main.xlsx
counter: 9
file_name: 009___422116_East_Palo_Alto___main.xlsx
counter: 10
file_name: 010___403059_East_Palo_Alto___off.xlsx
counter: 11
file_name: 011___403060_East_Palo_Alto___on.xlsx
counter: 12
file_name: 012___400859_East_Palo_Alto___main.xlsx
counter: 13
file_name: 013___403061_East_Palo_Alto___on.xlsx
counter: 14
file_name: 014___404533_East_Palo_Alto___main.xlsx
counter: 15
file_name: 015___408267_East_Palo_Alto___on.xlsx
counter: 16
file_name: 016___400981_East_Palo_Alto___main.xlsx
counter: 17
file_name: 017___402398_East_Palo_

In [9]:
# directory_path = '../data/pems_flow___101_N_Sep_to_Nov_2024/'
# file_name_list = [f for f in os.listdir(directory_path) \
#                   if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x']
# file_name_list.sort()

# file_name_list



# Scratch Work

In [ ]:
# date_number_list = ['03', '04', '05', '06', '09', '10', '11', '12', '13', '16', \
#                     '17', '18', '19', '20', '23', '24', '25', '26', '27', '30']
# hours_list = ['06', '07', '08', '09', '10', '11', '12']

# times_dict = {}
# counter = 0
# for date_number in date_number_list:
# #     print("date_number:", date_number)
#     for hour in hours_list:
# #         print("hour:", hour)
#         times_dict[counter] = '2024-09-' + date_number + ' ' + hour + ':00:00'
#         counter += 1

# dict_data = {}
# dict_data["Times / Type"] = list(times_dict.values())



# dict_data["Times / Type"]


In [ ]:
# list_1 = [5]
# list_1 += [3]
# list_1